In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
#필요한 라이브러리 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

plt.rcParams['axes.unicode_minus'] = False

In [57]:
df=pd.read_excel('C:/Users/user/Downloads/맛집리스트_주소추가.xlsx')
df

,음식점명,응답자 성별,맛집 카테고리,방문 목적,최애 메뉴 (복수 입력 가능),맛집 평점,키워드1,키워드2,키워드3,키워드4,키워드5,평점_네이버,방문자 리뷰수,블로그 리뷰수,음식 카테고리,주소
0,알부자,남자,술,친구들과의 모임,알찜,5.00,"""음식이 맛있어요""","""양이 많아요""","""재료가 신선해요""","""친절해요""","""특별한 메뉴가 있어요""",4.38,"3,286","1,442","해물,생선요리",주소보기\n서울특별시 강남구 강남대로98길 14 1층 알부자
1,야마야을지로,여자,술,친구들과의 모임,"가라아게, 모츠나베",5.00,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 청결해요""","""재료가 신선해요""",4.37,641,439,일식당,주소보기\n서울특별시 중구 을지로5길 19 페럼타워 지하1층 야마야 페럼타워점
2,세월마차,남자,술,친구들과의 모임,"오돌뼈, 계란전",5.00,"""기본 안주가 좋아요""","""음식이 맛있어요""","""친절해요""","""가성비가 좋아요""","""특별한 메뉴가 있어요""",4.53,123,110,포장마차,주소보기\n서울특별시 관악구 은천로 32
3,하쿠텐라멘,남자,밥,데이트,매운 이에케 라멘,4.20,"""음식이 맛있어요""","""친절해요""","""혼밥하기 좋아요""","""특별한 메뉴가 있어요""","""양이 많아요""",4.58,310,311,일본식라면,주소보기\n서울특별시 마포구 동교로 266-12 반지하 왼쪽문
4,떡도리탕강남본점,여자,밥,친구들과의 모임,떡도리탕,5.00,"""음식이 맛있어요""","""양이 많아요""","""친절해요""","""특별한 메뉴가 있어요""","""매장이 넓어요""",4.47,"1,486","1,883",닭볶음탕,주소보기\n서울특별시 강남구 테헤란로1길 28-9 1층
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,강남명전,남자,술,친구들과의 모임,"치즈감자전, 마늘수육&보쌈김치",4.50,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 넓어요""","""단체모임 하기 좋아요""",4.31,605,541,"전,빈대떡",주소보기\n서울특별시 서초구 서초대로75길 36 대청빌딩 2층
104,호요,여자,술,데이트,"연어 사시미, 히로시마풍 오꼬노미야끼",4.45,"""음식이 맛있어요""","""술이 다양해요""","""친절해요""","""특별한 메뉴가 있어요""","""기본 안주가 좋아요""",4.6,530,506,이자카야,주소보기\n서울특별시 동작구 동작대로13길 6-7 1층 호요
105,봉고기,남자,밥,친구들과의 모임,등심덧살,4.35,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""매장이 청결해요""","""매장이 넓어요""",4.54,"2,872",842,"육류,고기요리",주소보기\n서울특별시 구로구 경인로 619-3 푸르지오 3차 아파트 상가 1층
106,담산,남자,밥,데이트,양푼등갈비찜,4.50,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""양이 많아요""","""특별한 메뉴가 있어요""",4.59,"1,658","1,270",한식,주소보기\n서울특별시 서대문구 연세로5다길 5 1층 담산 신촌본점


# 데이터 전처리
- 방문자 리뷰수 & 블로그 리뷰수 -> ',' 문자열 제거


In [58]:
df['방문자 리뷰수'] = df['방문자 리뷰수'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
df['블로그 리뷰수'] = df['블로그 리뷰수'].str.replace(pat=r'[^\w]', repl=r'', regex=True)

In [59]:
df['추천메뉴'] = df['최애 메뉴 (복수 입력 가능)']
df['최애 메뉴 (복수 입력 가능)'] = df['최애 메뉴 (복수 입력 가능)'].str.replace(pat=r'[^\w]', repl=r'', regex=True)

- 네이버 평점 'x'값을 가지는 곳에 0 대체 후 null값 처리

In [60]:
df['평점_네이버'] = df['평점_네이버'].replace('x', '0')
df['평점_네이버'] = df['평점_네이버'].astype(float)

In [61]:
#평점이 없는 음식점의 index값 추출
평점x_index = df[df['평점_네이버'] == 0.0].index.tolist()
for i in 평점x_index :
    df.loc[i, '평점_네이버'] = np.nan

In [62]:
df[df['평점_네이버'].isna() == True]

,음식점명,응답자 성별,맛집 카테고리,방문 목적,최애 메뉴 (복수 입력 가능),맛집 평점,키워드1,키워드2,키워드3,키워드4,키워드5,평점_네이버,방문자 리뷰수,블로그 리뷰수,음식 카테고리,주소,추천메뉴
7,낭만짚불구이,남자,술,데이트,아보카도육회명란구이짚불닭,4.50,"""음식이 맛있어요""","""친절해요""","""가성비가 좋아요""","""인테리어가 멋져요""","""매장이 넓어요""",NaN,44,9,"육류,고기요리",주소보기\n서울특별시 송파구 백제고분로7길 28-11 영동빌딩 1층,"아보카도육회, 명란구이, 짚불닭"
10,서울낙업,여자,술,친구들과의 모임,생연어사시미블랙토리가라아게차돌박이참께나베,4.00,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 청결해요""","""인테리어가 멋져요""",NaN,62,112,일식당,주소보기\n서울특별시 광진구 동일로22길 117-23 지하 1층,"생연어 사시미, 블랙 토리 가라아게, 차돌박이 참께나베"
18,777포차,남자,술,친구들과의 모임,옛날도시락,4.50,"""음식이 맛있어요""","""가성비가 좋아요""","""매장이 넓어요""","""인테리어가 멋져요""","""친절해요""",NaN,16,4,한식,주소보기\n서울특별시 강동구 양재대로89가길 42 1층 1층(성내동),옛날도시락
33,동도참치초밥,남자,밥,친구들과의 모임,회덮밥알밥,4.20,"""음식이 맛있어요""","""재료가 신선해요""","""가성비가 좋아요""","""친절해요""","""양이 많아요""",NaN,98,16,생선회,주소보기\n서울특별시 서초구 방배중앙로 191 방배빌딩 1층 나호,"회덮밥, 알밥"
34,생생집,여자,술,회식(회사/동아리),급랭오리광주식들깨오리탕,4.15,"""음식이 맛있어요""","""친절해요""","""매장이 청결해요""","""인테리어가 멋져요""","""특별한 메뉴가 있어요""",NaN,153,120,오리요리,주소보기\n서울특별시 동작구 동작대로5길 16 1층,"급랭오리, 광주식 들깨오리탕"
36,진저베어,여자,카페,데이트,클래식미트파이초콜릿무스파이,4.00,"""디저트가 맛있어요""","""특별한 메뉴가 있어요""","""인테리어가 멋져요""","""커피가 맛있어요""","""음료가 맛있어요""",NaN,2773,1975,"카페,디저트",주소보기\n서울특별시 송파구 백제고분로41길 43-7 1층,"클래식미트파이, 초콜릿무스파이"
37,애정마라탕,여자,밥,친구들과의 모임,마라샹궈,5.00,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""매장이 넓어요""","""매장이 청결해요""",NaN,2388,1495,중식당,주소보기\n서울특별시 성북구 동소문로20나길 11,마라샹궈
38,88갈비,여자,밥,친구들과의 모임,88갈비그시절김치볶음밥,4.60,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""매장이 청결해요""","""인테리어가 멋져요""",NaN,50,49,"육류,고기요리",주소보기\n서울특별시 관악구 조원중앙로1길 46 1층,"88갈비, 그 시절 김치볶음밥"
45,범맥주건대점,여자,술,친구들과의 모임,범꽃맥주꼬치꽂힌닭,4.88,"""음식이 맛있어요""","""술이 다양해요""","""음악이 좋아요""","""친절해요""","""인테리어가 멋져요""",NaN,436,357,요리주점,주소보기\n서울특별시 광진구 아차산로29길 8 2층~4층,"범꽃맥주, 꼬치꽂힌닭"
49,김군네짬뽕,남자,밥,친구들과의 모임,짬뽕,4.70,"""음식이 맛있어요""","""양이 많아요""","""가성비가 좋아요""","""재료가 신선해요""","""특별한 메뉴가 있어요""",NaN,41,12,중식당,주소보기\n서울특별시 강북구 오패산로30길 13,짬뽕


- 주소 : '구' 만 추출

In [63]:
지역_list = []
주소_list = df['주소'].str.replace('\n', ' ').str.split(' ')

for i in range(len(주소_list)):
    지역_list.append(주소_list[i][2])

In [64]:
df['지역'] = 지역_list
df

,음식점명,응답자 성별,맛집 카테고리,방문 목적,최애 메뉴 (복수 입력 가능),맛집 평점,키워드1,키워드2,키워드3,키워드4,키워드5,평점_네이버,방문자 리뷰수,블로그 리뷰수,음식 카테고리,주소,추천메뉴,지역
0,알부자,남자,술,친구들과의 모임,알찜,5.00,"""음식이 맛있어요""","""양이 많아요""","""재료가 신선해요""","""친절해요""","""특별한 메뉴가 있어요""",4.38,3286,1442,"해물,생선요리",주소보기\n서울특별시 강남구 강남대로98길 14 1층 알부자,알찜,강남구
1,야마야을지로,여자,술,친구들과의 모임,가라아게모츠나베,5.00,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 청결해요""","""재료가 신선해요""",4.37,641,439,일식당,주소보기\n서울특별시 중구 을지로5길 19 페럼타워 지하1층 야마야 페럼타워점,"가라아게, 모츠나베",중구
2,세월마차,남자,술,친구들과의 모임,오돌뼈계란전,5.00,"""기본 안주가 좋아요""","""음식이 맛있어요""","""친절해요""","""가성비가 좋아요""","""특별한 메뉴가 있어요""",4.53,123,110,포장마차,주소보기\n서울특별시 관악구 은천로 32,"오돌뼈, 계란전",관악구
3,하쿠텐라멘,남자,밥,데이트,매운이에케라멘,4.20,"""음식이 맛있어요""","""친절해요""","""혼밥하기 좋아요""","""특별한 메뉴가 있어요""","""양이 많아요""",4.58,310,311,일본식라면,주소보기\n서울특별시 마포구 동교로 266-12 반지하 왼쪽문,매운 이에케 라멘,마포구
4,떡도리탕강남본점,여자,밥,친구들과의 모임,떡도리탕,5.00,"""음식이 맛있어요""","""양이 많아요""","""친절해요""","""특별한 메뉴가 있어요""","""매장이 넓어요""",4.47,1486,1883,닭볶음탕,주소보기\n서울특별시 강남구 테헤란로1길 28-9 1층,떡도리탕,강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,강남명전,남자,술,친구들과의 모임,치즈감자전마늘수육보쌈김치,4.50,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 넓어요""","""단체모임 하기 좋아요""",4.31,605,541,"전,빈대떡",주소보기\n서울특별시 서초구 서초대로75길 36 대청빌딩 2층,"치즈감자전, 마늘수육&보쌈김치",서초구
104,호요,여자,술,데이트,연어사시미히로시마풍오꼬노미야끼,4.45,"""음식이 맛있어요""","""술이 다양해요""","""친절해요""","""특별한 메뉴가 있어요""","""기본 안주가 좋아요""",4.60,530,506,이자카야,주소보기\n서울특별시 동작구 동작대로13길 6-7 1층 호요,"연어 사시미, 히로시마풍 오꼬노미야끼",동작구
105,봉고기,남자,밥,친구들과의 모임,등심덧살,4.35,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""매장이 청결해요""","""매장이 넓어요""",4.54,2872,842,"육류,고기요리",주소보기\n서울특별시 구로구 경인로 619-3 푸르지오 3차 아파트 상가 1층,등심덧살,구로구
106,담산,남자,밥,데이트,양푼등갈비찜,4.50,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""양이 많아요""","""특별한 메뉴가 있어요""",4.59,1658,1270,한식,주소보기\n서울특별시 서대문구 연세로5다길 5 1층 담산 신촌본점,양푼등갈비찜,서대문구


- 음식 카테고리2 변수 생성
    - 추후 코사인 유사도를 구하기 위한 기준 변수가 될 것이기 때문에 불필요한 공백, 문자열 제거가 필요

In [ ]:
df['음식 카테고리2'] = df['음식 카테고리']
df['음식 카테고리2'] = df['음식 카테고리2'].str.replace(pat=r'[^\w]', repl=r'', regex=True)

- 방문목적 변수에서 문자열 제거

In [ ]:
df['방문 목적'] = df['방문 목적'].str.replace(' ','')

# 파생변수 생성
- 음식 카테고리 -> (한식, 중식, 양식, 일식, 카페, 분식, 아시아, 술) 재분류

In [65]:
#음식 카테고리 정리
df['음식 카테고리'].unique()

array(['해물,생선요리', '일식당', '포장마차', '일본식라면', '닭볶음탕', '종합분식', '육류,고기요리', '한식',
       '카페,디저트', '낙지요리', '칼국수,만두', '아시아음식', '생선회', '스파게티,파스타전문', '중식당',
       '딤섬,중식만두', '양식', '닭요리', '돼지고기구이', '오리요리', '베트남음식', '전통,민속주점',
       '요리주점', '바(BAR)', '와인', '퓨전음식', '초밥,롤', '이탈리아음식', '곱창,막창,양',
       '이자카야', '돈가스', '덮밥', '닭갈비', '테이크아웃커피', '곰탕,설렁탕', '순대,순댓국', '닭발',
       '맥주,호프', '족발,보쌈', '불닭', '피자', '카페', '전,빈대떡'], dtype=object)

In [66]:
한식 = ['해물,생선요리', '닭볶음탕', '육류,고기요리', '한식', '낙지요리', '닭요리', '돼지고기구이', '오리요리', '곱창,막창,양', '닭갈비', '곰탕,설렁탕', '순대,순댓국', '닭발', '불닭', '족발,보쌈', '돈가스']
분식 = ['종합분식', '칼국수,만두']
기타 = ['아시아음식', '베트남음식', '퓨전음식']
술 = ['포장마차', '요리주점', '바(BAR)', '와인', '이자카야', '맥주,호프', '전,빈대떡', '전통,민속주점']
일식 = ['일식당', '일본식라면', '초밥,롤', '덮밥', '생선회']
카페 = ['카페,디저트', '카페', '테이크아웃커피']
양식 = ['이탈리아음식', '피자', '스파게티,파스타전문', '양식']
중식 = ['중식당', '딤섬,중식만두']


In [67]:
df['식당_카테고리'] = ""

In [68]:
def change_cat(df, list_x, name) :
    for i in range(len(df)) :
        if df['음식 카테고리'][i] in list_x :
            df['식당_카테고리'][i] = name 
        else :
            pass

In [69]:
change_cat(df, 한식, '한식')
change_cat(df, 기타, '기타')
change_cat(df, 분식, '분식')
change_cat(df, 술, '술집')
change_cat(df, 일식, '일식')
change_cat(df, 카페, '카페')
change_cat(df, 양식, '양식')
change_cat(df, 중식, '중식')

In [70]:
df

,음식점명,응답자 성별,맛집 카테고리,방문 목적,최애 메뉴 (복수 입력 가능),맛집 평점,키워드1,키워드2,키워드3,키워드4,키워드5,평점_네이버,방문자 리뷰수,블로그 리뷰수,음식 카테고리,주소,추천메뉴,지역,식당_카테고리
0,알부자,남자,술,친구들과의 모임,알찜,5.00,"""음식이 맛있어요""","""양이 많아요""","""재료가 신선해요""","""친절해요""","""특별한 메뉴가 있어요""",4.38,3286,1442,"해물,생선요리",주소보기\n서울특별시 강남구 강남대로98길 14 1층 알부자,알찜,강남구,한식
1,야마야을지로,여자,술,친구들과의 모임,가라아게모츠나베,5.00,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 청결해요""","""재료가 신선해요""",4.37,641,439,일식당,주소보기\n서울특별시 중구 을지로5길 19 페럼타워 지하1층 야마야 페럼타워점,"가라아게, 모츠나베",중구,일식
2,세월마차,남자,술,친구들과의 모임,오돌뼈계란전,5.00,"""기본 안주가 좋아요""","""음식이 맛있어요""","""친절해요""","""가성비가 좋아요""","""특별한 메뉴가 있어요""",4.53,123,110,포장마차,주소보기\n서울특별시 관악구 은천로 32,"오돌뼈, 계란전",관악구,술집
3,하쿠텐라멘,남자,밥,데이트,매운이에케라멘,4.20,"""음식이 맛있어요""","""친절해요""","""혼밥하기 좋아요""","""특별한 메뉴가 있어요""","""양이 많아요""",4.58,310,311,일본식라면,주소보기\n서울특별시 마포구 동교로 266-12 반지하 왼쪽문,매운 이에케 라멘,마포구,일식
4,떡도리탕강남본점,여자,밥,친구들과의 모임,떡도리탕,5.00,"""음식이 맛있어요""","""양이 많아요""","""친절해요""","""특별한 메뉴가 있어요""","""매장이 넓어요""",4.47,1486,1883,닭볶음탕,주소보기\n서울특별시 강남구 테헤란로1길 28-9 1층,떡도리탕,강남구,한식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,강남명전,남자,술,친구들과의 모임,치즈감자전마늘수육보쌈김치,4.50,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 넓어요""","""단체모임 하기 좋아요""",4.31,605,541,"전,빈대떡",주소보기\n서울특별시 서초구 서초대로75길 36 대청빌딩 2층,"치즈감자전, 마늘수육&보쌈김치",서초구,술집
104,호요,여자,술,데이트,연어사시미히로시마풍오꼬노미야끼,4.45,"""음식이 맛있어요""","""술이 다양해요""","""친절해요""","""특별한 메뉴가 있어요""","""기본 안주가 좋아요""",4.60,530,506,이자카야,주소보기\n서울특별시 동작구 동작대로13길 6-7 1층 호요,"연어 사시미, 히로시마풍 오꼬노미야끼",동작구,술집
105,봉고기,남자,밥,친구들과의 모임,등심덧살,4.35,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""매장이 청결해요""","""매장이 넓어요""",4.54,2872,842,"육류,고기요리",주소보기\n서울특별시 구로구 경인로 619-3 푸르지오 3차 아파트 상가 1층,등심덧살,구로구,한식
106,담산,남자,밥,데이트,양푼등갈비찜,4.50,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""양이 많아요""","""특별한 메뉴가 있어요""",4.59,1658,1270,한식,주소보기\n서울특별시 서대문구 연세로5다길 5 1층 담산 신촌본점,양푼등갈비찜,서대문구,한식


In [71]:
df['식당_카테고리'].unique()

array(['한식', '일식', '술집', '분식', '카페', '기타', '양식', '중식'], dtype=object)

# 평점 컬럼 생성
- 네이버_평점 결측치 -> 식당 카테고리별 평균값과 지인 평점의 평균으로 대치

In [ ]:
df_평점['평점_네이버'][평균평점_idx] + df['맛집 평점'][i]) / 2

In [72]:
df

,음식점명,응답자 성별,맛집 카테고리,방문 목적,최애 메뉴 (복수 입력 가능),맛집 평점,키워드1,키워드2,키워드3,키워드4,키워드5,평점_네이버,방문자 리뷰수,블로그 리뷰수,음식 카테고리,주소,추천메뉴,지역,식당_카테고리
0,알부자,남자,술,친구들과의 모임,알찜,5.00,"""음식이 맛있어요""","""양이 많아요""","""재료가 신선해요""","""친절해요""","""특별한 메뉴가 있어요""",4.38,3286,1442,"해물,생선요리",주소보기\n서울특별시 강남구 강남대로98길 14 1층 알부자,알찜,강남구,한식
1,야마야을지로,여자,술,친구들과의 모임,가라아게모츠나베,5.00,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 청결해요""","""재료가 신선해요""",4.37,641,439,일식당,주소보기\n서울특별시 중구 을지로5길 19 페럼타워 지하1층 야마야 페럼타워점,"가라아게, 모츠나베",중구,일식
2,세월마차,남자,술,친구들과의 모임,오돌뼈계란전,5.00,"""기본 안주가 좋아요""","""음식이 맛있어요""","""친절해요""","""가성비가 좋아요""","""특별한 메뉴가 있어요""",4.53,123,110,포장마차,주소보기\n서울특별시 관악구 은천로 32,"오돌뼈, 계란전",관악구,술집
3,하쿠텐라멘,남자,밥,데이트,매운이에케라멘,4.20,"""음식이 맛있어요""","""친절해요""","""혼밥하기 좋아요""","""특별한 메뉴가 있어요""","""양이 많아요""",4.58,310,311,일본식라면,주소보기\n서울특별시 마포구 동교로 266-12 반지하 왼쪽문,매운 이에케 라멘,마포구,일식
4,떡도리탕강남본점,여자,밥,친구들과의 모임,떡도리탕,5.00,"""음식이 맛있어요""","""양이 많아요""","""친절해요""","""특별한 메뉴가 있어요""","""매장이 넓어요""",4.47,1486,1883,닭볶음탕,주소보기\n서울특별시 강남구 테헤란로1길 28-9 1층,떡도리탕,강남구,한식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,강남명전,남자,술,친구들과의 모임,치즈감자전마늘수육보쌈김치,4.50,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 넓어요""","""단체모임 하기 좋아요""",4.31,605,541,"전,빈대떡",주소보기\n서울특별시 서초구 서초대로75길 36 대청빌딩 2층,"치즈감자전, 마늘수육&보쌈김치",서초구,술집
104,호요,여자,술,데이트,연어사시미히로시마풍오꼬노미야끼,4.45,"""음식이 맛있어요""","""술이 다양해요""","""친절해요""","""특별한 메뉴가 있어요""","""기본 안주가 좋아요""",4.60,530,506,이자카야,주소보기\n서울특별시 동작구 동작대로13길 6-7 1층 호요,"연어 사시미, 히로시마풍 오꼬노미야끼",동작구,술집
105,봉고기,남자,밥,친구들과의 모임,등심덧살,4.35,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""매장이 청결해요""","""매장이 넓어요""",4.54,2872,842,"육류,고기요리",주소보기\n서울특별시 구로구 경인로 619-3 푸르지오 3차 아파트 상가 1층,등심덧살,구로구,한식
106,담산,남자,밥,데이트,양푼등갈비찜,4.50,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""양이 많아요""","""특별한 메뉴가 있어요""",4.59,1658,1270,한식,주소보기\n서울특별시 서대문구 연세로5다길 5 1층 담산 신촌본점,양푼등갈비찜,서대문구,한식


In [73]:
# 식당_카테고리별 음식점 평균값
df_평점 = pd.DataFrame(df.groupby('식당_카테고리')['평점_네이버'].mean()).round(2).reset_index()
df_평점

,식당_카테고리,평점_네이버
0,기타,4.57
1,분식,4.34
2,술집,4.56
3,양식,4.61
4,일식,4.52
5,중식,4.48
6,카페,4.51
7,한식,4.49


In [74]:
#평점이 없는 음식점의 index값 추출
평점x_index = df[df['평점_네이버'].isna() == True].index.tolist()

In [75]:
#식당 카테고리별 평균 평점 & 지인 평점의 평균으로 결측치 처리
for i in 평점x_index :
    평균평점_idx = df_평점[df_평점['식당_카테고리'] == df.loc[i][-1]].index[0]
    df['평점_네이버'][i] = round((df_평점['평점_네이버'][평균평점_idx] + df['맛집 평점'][i]) / 2,2)
    

In [76]:
df.loc[평점x_index]

,음식점명,응답자 성별,맛집 카테고리,방문 목적,최애 메뉴 (복수 입력 가능),맛집 평점,키워드1,키워드2,키워드3,키워드4,키워드5,평점_네이버,방문자 리뷰수,블로그 리뷰수,음식 카테고리,주소,추천메뉴,지역,식당_카테고리
7,낭만짚불구이,남자,술,데이트,아보카도육회명란구이짚불닭,4.50,"""음식이 맛있어요""","""친절해요""","""가성비가 좋아요""","""인테리어가 멋져요""","""매장이 넓어요""",4.50,44,9,"육류,고기요리",주소보기\n서울특별시 송파구 백제고분로7길 28-11 영동빌딩 1층,"아보카도육회, 명란구이, 짚불닭",송파구,한식
10,서울낙업,여자,술,친구들과의 모임,생연어사시미블랙토리가라아게차돌박이참께나베,4.00,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 청결해요""","""인테리어가 멋져요""",4.26,62,112,일식당,주소보기\n서울특별시 광진구 동일로22길 117-23 지하 1층,"생연어 사시미, 블랙 토리 가라아게, 차돌박이 참께나베",광진구,일식
18,777포차,남자,술,친구들과의 모임,옛날도시락,4.50,"""음식이 맛있어요""","""가성비가 좋아요""","""매장이 넓어요""","""인테리어가 멋져요""","""친절해요""",4.50,16,4,한식,주소보기\n서울특별시 강동구 양재대로89가길 42 1층 1층(성내동),옛날도시락,강동구,한식
33,동도참치초밥,남자,밥,친구들과의 모임,회덮밥알밥,4.20,"""음식이 맛있어요""","""재료가 신선해요""","""가성비가 좋아요""","""친절해요""","""양이 많아요""",4.36,98,16,생선회,주소보기\n서울특별시 서초구 방배중앙로 191 방배빌딩 1층 나호,"회덮밥, 알밥",서초구,일식
34,생생집,여자,술,회식(회사/동아리),급랭오리광주식들깨오리탕,4.15,"""음식이 맛있어요""","""친절해요""","""매장이 청결해요""","""인테리어가 멋져요""","""특별한 메뉴가 있어요""",4.32,153,120,오리요리,주소보기\n서울특별시 동작구 동작대로5길 16 1층,"급랭오리, 광주식 들깨오리탕",동작구,한식
36,진저베어,여자,카페,데이트,클래식미트파이초콜릿무스파이,4.00,"""디저트가 맛있어요""","""특별한 메뉴가 있어요""","""인테리어가 멋져요""","""커피가 맛있어요""","""음료가 맛있어요""",4.26,2773,1975,"카페,디저트",주소보기\n서울특별시 송파구 백제고분로41길 43-7 1층,"클래식미트파이, 초콜릿무스파이",송파구,카페
37,애정마라탕,여자,밥,친구들과의 모임,마라샹궈,5.00,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""매장이 넓어요""","""매장이 청결해요""",4.74,2388,1495,중식당,주소보기\n서울특별시 성북구 동소문로20나길 11,마라샹궈,성북구,중식
38,88갈비,여자,밥,친구들과의 모임,88갈비그시절김치볶음밥,4.60,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""매장이 청결해요""","""인테리어가 멋져요""",4.54,50,49,"육류,고기요리",주소보기\n서울특별시 관악구 조원중앙로1길 46 1층,"88갈비, 그 시절 김치볶음밥",관악구,한식
45,범맥주건대점,여자,술,친구들과의 모임,범꽃맥주꼬치꽂힌닭,4.88,"""음식이 맛있어요""","""술이 다양해요""","""음악이 좋아요""","""친절해요""","""인테리어가 멋져요""",4.72,436,357,요리주점,주소보기\n서울특별시 광진구 아차산로29길 8 2층~4층,"범꽃맥주, 꼬치꽂힌닭",광진구,술집
49,김군네짬뽕,남자,밥,친구들과의 모임,짬뽕,4.70,"""음식이 맛있어요""","""양이 많아요""","""가성비가 좋아요""","""재료가 신선해요""","""특별한 메뉴가 있어요""",4.59,41,12,중식당,주소보기\n서울특별시 강북구 오패산로30길 13,짬뽕,강북구,중식


# CountVectorizer
- 아이템 기반 추천 시스템을 구축할 때 필요
- 음식점 카테고리, 메뉴, 음식 범주 -> 총 3개의 변수를 기반으로 음식점 간 유사도 산출

In [385]:
re_list = ['최애 메뉴 (복수 입력 가능)', '식당_카테고리', '음식 카테고리2']
len(re_list)

3

In [386]:
#select한 변수를 모두 포함하는 새로운 컬럼 생성
def recommendation_countvectorize(row) : 
    return str(row[re_list[0]])+ " " + str(row[re_list[1]]) + " " + str(row[re_list[2]]) 

In [387]:
df['recommendation_data'] = df.apply(recommendation_countvectorize, axis=1)
df

,음식점명,응답자 성별,맛집 카테고리,방문 목적,최애 메뉴 (복수 입력 가능),맛집 평점,키워드1,키워드2,키워드3,키워드4,...,평점_네이버,방문자 리뷰수,블로그 리뷰수,음식 카테고리,주소,추천메뉴,지역,식당_카테고리,음식 카테고리2,recommendation_data
0,알부자,남자,술,친구들과의모임,알찜,5.00,"""음식이 맛있어요""","""양이 많아요""","""재료가 신선해요""","""친절해요""",...,4.38,3286,1442,"해물,생선요리",주소보기\n서울특별시 강남구 강남대로98길 14 1층 알부자,알찜,강남구,한식,해물생선요리,알찜 한식 해물생선요리
1,야마야을지로,여자,술,친구들과의모임,가라아게모츠나베,5.00,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 청결해요""",...,4.37,641,439,일식당,주소보기\n서울특별시 중구 을지로5길 19 페럼타워 지하1층 야마야 페럼타워점,"가라아게, 모츠나베",중구,일식,일식당,가라아게모츠나베 일식 일식당
2,세월마차,남자,술,친구들과의모임,오돌뼈계란전,5.00,"""기본 안주가 좋아요""","""음식이 맛있어요""","""친절해요""","""가성비가 좋아요""",...,4.53,123,110,포장마차,주소보기\n서울특별시 관악구 은천로 32,"오돌뼈, 계란전",관악구,술집,포장마차,오돌뼈계란전 술집 포장마차
3,하쿠텐라멘,남자,밥,데이트,매운이에케라멘,4.20,"""음식이 맛있어요""","""친절해요""","""혼밥하기 좋아요""","""특별한 메뉴가 있어요""",...,4.58,310,311,일본식라면,주소보기\n서울특별시 마포구 동교로 266-12 반지하 왼쪽문,매운 이에케 라멘,마포구,일식,일본식라면,매운이에케라멘 일식 일본식라면
4,떡도리탕강남본점,여자,밥,친구들과의모임,떡도리탕,5.00,"""음식이 맛있어요""","""양이 많아요""","""친절해요""","""특별한 메뉴가 있어요""",...,4.47,1486,1883,닭볶음탕,주소보기\n서울특별시 강남구 테헤란로1길 28-9 1층,떡도리탕,강남구,한식,닭볶음탕,떡도리탕 한식 닭볶음탕
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,강남명전,남자,술,친구들과의모임,치즈감자전마늘수육보쌈김치,4.50,"""음식이 맛있어요""","""특별한 메뉴가 있어요""","""친절해요""","""매장이 넓어요""",...,4.31,605,541,"전,빈대떡",주소보기\n서울특별시 서초구 서초대로75길 36 대청빌딩 2층,"치즈감자전, 마늘수육&보쌈김치",서초구,술집,전빈대떡,치즈감자전마늘수육보쌈김치 술집 전빈대떡
104,호요,여자,술,데이트,연어사시미히로시마풍오꼬노미야끼,4.45,"""음식이 맛있어요""","""술이 다양해요""","""친절해요""","""특별한 메뉴가 있어요""",...,4.60,530,506,이자카야,주소보기\n서울특별시 동작구 동작대로13길 6-7 1층 호요,"연어 사시미, 히로시마풍 오꼬노미야끼",동작구,술집,이자카야,연어사시미히로시마풍오꼬노미야끼 술집 이자카야
105,봉고기,남자,밥,친구들과의모임,등심덧살,4.35,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""매장이 청결해요""",...,4.54,2872,842,"육류,고기요리",주소보기\n서울특별시 구로구 경인로 619-3 푸르지오 3차 아파트 상가 1층,등심덧살,구로구,한식,육류고기요리,등심덧살 한식 육류고기요리
106,담산,남자,밥,데이트,양푼등갈비찜,4.50,"""음식이 맛있어요""","""친절해요""","""재료가 신선해요""","""양이 많아요""",...,4.59,1658,1270,한식,주소보기\n서울특별시 서대문구 연세로5다길 5 1층 담산 신촌본점,양푼등갈비찜,서대문구,한식,한식,양푼등갈비찜 한식 한식


In [388]:
count_vect = CountVectorizer(ngram_range=(1, 3))
count_matrix = count_vect.fit_transform(df['recommendation_data'])

In [389]:
print(count_matrix.shape)

(108, 412)


In [390]:
count_vect.vocabulary_

{'알찜': 242,
 '한식': 382,
 '해물생선요리': 402,
 '알찜 한식': 243,
 '한식 해물생선요리': 398,
 '알찜 한식 해물생선요리': 244,
 '가라아게모츠나베': 6,
 '일식': 299,
 '일식당': 305,
 '가라아게모츠나베 일식': 7,
 '일식 일식당': 303,
 '가라아게모츠나베 일식 일식당': 8,
 '오돌뼈계란전': 274,
 '술집': 205,
 '포장마차': 376,
 '오돌뼈계란전 술집': 275,
 '술집 포장마차': 213,
 '오돌뼈계란전 술집 포장마차': 276,
 '매운이에케라멘': 121,
 '일본식라면': 298,
 '매운이에케라멘 일식': 122,
 '일식 일본식라면': 302,
 '매운이에케라멘 일식 일본식라면': 123,
 '떡도리탕': 103,
 '닭볶음탕': 62,
 '떡도리탕 한식': 104,
 '한식 닭볶음탕': 388,
 '떡도리탕 한식 닭볶음탕': 105,
 '치즈밥': 339,
 '분식': 154,
 '종합분식': 315,
 '치즈밥 분식': 340,
 '분식 종합분식': 155,
 '치즈밥 분식 종합분식': 341,
 '안심카츠': 231,
 '안심카츠 일식': 232,
 '안심카츠 일식 일식당': 233,
 '아보카도육회명란구이짚불닭': 227,
 '육류고기요리': 292,
 '아보카도육회명란구이짚불닭 한식': 228,
 '한식 육류고기요리': 395,
 '아보카도육회명란구이짚불닭 한식 육류고기요리': 229,
 '순대모듬내장탕': 201,
 '순대모듬내장탕 한식': 202,
 '한식 한식': 397,
 '순대모듬내장탕 한식 한식': 203,
 '기요한카이센동': 41,
 '기요한카이센동 일식': 42,
 '기요한카이센동 일식 일식당': 43,
 '생연어사시미블랙토리가라아게차돌박이참께나베': 186,
 '생연어사시미블랙토리가라아게차돌박이참께나베 일식': 187,
 '생연어사시미블랙토리가라아게차돌박이참께나베 일식 일식당': 188,
 '크로플': 354,
 '카페': 345

In [391]:
# 코사인 유사도 계산
food_sim = cosine_similarity(count_matrix, count_matrix)
print(food_sim.shape)

(108, 108)


In [405]:
# 첫번째 음식점과 모든 음식점 간의 유사도 값
food_sim[0]

array([1.        , 0.        , 0.        , 0.        , 0.16666667,
       0.        , 0.        , 0.16666667, 0.28867513, 0.        ,
       0.        , 0.        , 0.16666667, 0.        , 0.16666667,
       0.        , 0.        , 0.        , 0.28867513, 0.28867513,
       0.        , 0.28867513, 0.        , 0.        , 0.        ,
       0.        , 0.16666667, 0.        , 0.16666667, 0.16666667,
       0.16666667, 0.        , 0.        , 0.        , 0.16666667,
       0.16666667, 0.        , 0.        , 0.16666667, 0.16666667,
       0.28867513, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.16666667, 0.        , 0.16666667, 0.        ,
       0.14433757, 0.        , 0.        , 0.16666667, 0.14433757,
       0.16666667, 0.        , 0.        , 0.16666667, 0.        ,
       0.28867513, 0.16666667, 0.        , 0.        , 0.     

In [393]:
#argsort 배열 내림차순 정렬 -> 유사도가 높을수록 비슷한 음식점이기 때문 
food_sim_sorted_index = food_sim.argsort()[:, ::-1] 
print(food_sim_sorted_index[:2])

[[  0   8 106  70  21  19  18  40 101  28  77  76  68  65  63  26  58  99
   80  29  30 105  34  35  38  39  56  71  87  84  96   7   4  92  12  88
   86  75  14  64  60  22   3   6  15  37  36  41  33  32  31   9  11  10
    1  20   2  27  16  25   5  13  24  23  17 107  42  91  78  79  81  82
   83  85  89  90  93  43  94  95  97  98 100 102 103 104  74  73  72  69
   44  45  46  47  48  49  50  51  52  54  55  57  59  61  62  66  67  53]
 [  1  43  51  66  32   6  17   9  10  31  62  16  79  91  82  57  33  54
   52   3  35  37  36  34 107  38  39  40  30  42  44  45  46  41  25  29
   15   2   4   5   7   8  11  12  13  14  18  28  19  20  21  22  23  24
   48  26  27  47  53  49  81  84  85  86  87  88  89  90  92  93  94  95
   96  97  98  99 100 101 102 103 104 105  83  80  50  78 106  55  56  58
   59  60  61  63  64  65  67  68  69  70  71  72  73  74  75  76  77   0]]


# 추천시스템 구현

## 코사인 유사도 활용X
- 경험 기반 추천 시스템 
    - 수집한 데이터가 경험 기반의 맛집 데이터이기 때문에 방문목적, 음식 카테고리를 입력받아 지인 경험을 기반으로 음식점을 추출해 줄 수 있을 것
    - 방문목적, 맛집카테고리(밥, 카페, 술집 등), 방문지역 카테고리에 따라 랜덤으로 음식점을 샘플링 해줌

In [429]:
def find_sim_food1(df, sorted_index, visit, category, local):
    food = df.loc[(df['방문 목적']==visit) & (df['맛집 카테고리'] == category) & (df['지역'] == local)]
    df_select = df.iloc[food.index.values]
    return df_select.sample(n=3, replace=False)

In [446]:
food_recommendation = find_sim_food1(df, food_sim_sorted_index, '친구들과의모임', '밥', '강남구')
food_recommendation[['음식점명', '평점_네이버', '추천메뉴', '식당_카테고리', '방문 목적', '맛집 카테고리']]

,음식점명,평점_네이버,추천메뉴,식당_카테고리,방문 목적,맛집 카테고리
4,떡도리탕강남본점,4.47,떡도리탕,한식,친구들과의모임,밥
67,연남토마 강남점,4.57,가츠동,기타,친구들과의모임,밥
15,아시안라이브,4.49,"갈비구이, 된장찌개",기타,친구들과의모임,밥


## 코사인 유사도 활용O
- 아이템 기반 추천
    - 가본 음식점과 유사한 음식점을 가고 싶을 때 해당 추천 시스템을 활용할 수 있음

In [403]:
def find_sim_food2(df, sorted_index, food_name, top_n = 3):
    food = df.loc[df['음식점명']== food_name]
    food_index = food.index.values
    #print(food_index)
    similar_ind = sorted_index[food_index, : (top_n)]
    #print(similar_ind)
    similar_ind = similar_ind.reshape(-1)
    
    return df.iloc[similar_ind]

In [448]:
food_recommendation = find_sim_food2(df, food_sim_sorted_index, '애정마라탕')
food_recommendation[['음식점명', '평점_네이버', '추천메뉴', '식당_카테고리', '지역', '방문 목적']]

,음식점명,평점_네이버,추천메뉴,식당_카테고리,지역,방문 목적
37,애정마라탕,4.74,마라샹궈,중식,성북구,친구들과의모임
25,신사역 취향,4.32,간짜장,중식,서초구,친구들과의모임
49,김군네짬뽕,4.59,짬뽕,중식,강북구,친구들과의모임
